In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore
from datetime import datetime
from tqdm import tqdm

In [2]:
cred = credentials.Certificate("serviceAccount.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
matches_ref = db.collection('matches')
matches = {doc.id: doc.to_dict() for doc in matches_ref.stream()}

In [4]:
combined_data = []
for match_id, match_data in tqdm(matches.items(), desc="Processing matches"):
    commentary_ref = matches_ref.document(match_id).collection('commentary')
    for comm_doc in commentary_ref.stream():
        comm_data = comm_doc.to_dict()
        # Merge match data with commentary
        combined_entry = {
            **match_data,  # All match fields
            **comm_data,   # All commentary fields
            'commentary_id': comm_doc.id  # Keep commentary document ID
            }
        combined_data.append(combined_entry)

Processing matches:   0%|          | 0/1109 [00:00<?, ?it/s]

Processing matches: 100%|██████████| 1109/1109 [03:38<00:00,  5.06it/s]


In [5]:
df = pd.DataFrame(combined_data)

In [6]:
df.head(2)

,date_time,series_name,match_id,match_name,status,venue,season,commText,event,batsmanStrikerName,batTeamName,batsmanStrikerRuns,bowlerStrikerName,batTeamScore,inningsId,timestamp,ballNbr,commentary_id
0,"Apr 05, 08:00 PM","Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Cutting to Kedar Jadhav, B0$, sprays it down ...",NONE,Kedar Jadhav,RCB,5.0,Cutting,76.0,2.0,1.491412e+12,50.0,05n7e9Tn8XMYRx78imfr
1,"Apr 05, 08:00 PM","Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,B0$,NONE,Chahal,RCB,0.0,Bhuvneshwar,156.0,2.0,1.491416e+12,0.0,0i7aWf4ZSNalsSAuPps9


In [7]:
df['season'].unique()

array([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [8]:
len(matches.keys())

1109

In [9]:
df.match_id.nunique()

518

In [10]:
df.shape

(171432, 18)

In [11]:
df["batTeamName"].unique()

array(['RCB', 'SRH', nan, 'RPS', 'MI', 'KKR', 'GL', 'PBKS', 'DC', 'CSK',
       'RR', 'GT', 'LSG'], dtype=object)

In [12]:
df.isna().sum()

date_time               0
series_name             0
match_id                0
match_name              0
status                  0
venue                   0
season                  0
commText              518
event                 518
batsmanStrikerName    518
batTeamName           518
batsmanStrikerRuns    518
bowlerStrikerName     518
batTeamScore          518
inningsId             518
timestamp             518
ballNbr               518
commentary_id           0
dtype: int64

In [34]:
df1 = df.dropna()
df1.reset_index(drop=True, inplace=True)

In [35]:
df2 = df[df['ballNbr']!=0].sort_values(['season','match_id','inningsId','ballNbr'], ascending=True).reset_index(drop=True)

In [36]:
import pytz
df2["complete_datetime"] = pd.to_datetime(df2['timestamp'], unit='ms')
df2.head(3)

,date_time,series_name,match_id,match_name,status,venue,season,commText,event,batsmanStrikerName,batTeamName,batsmanStrikerRuns,bowlerStrikerName,batTeamScore,inningsId,timestamp,ballNbr,commentary_id,complete_datetime
0,"Apr 05, 08:00 PM","Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, no run, short of a len...",NONE,Warner,SRH,0.0,Tymal Mills,0.0,1.0,1.491403e+12,1.0,61sGXEURMeUQK18UtSoL,2017-04-05 14:32:22.378
1,"Apr 05, 08:00 PM","Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, no run, very full and ...",NONE,Warner,SRH,0.0,Tymal Mills,0.0,1.0,1.491403e+12,2.0,YMTy0sKwqCPHImS3CeB9,2017-04-05 14:32:56.547
2,"Apr 05, 08:00 PM","Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, B0$, B1$, sublime timin...",FOUR,Warner,SRH,4.0,Tymal Mills,4.0,1.0,1.491403e+12,3.0,EvnIsXWJaha8iF9MldoU,2017-04-05 14:33:36.255


In [37]:
df3 = df2.drop(['date_time', 'timestamp', 'commentary_id'], axis=1)

In [38]:
df3.shape

(124714, 16)

In [39]:
df3.groupby('batsmanStrikerName')['batsmanStrikerRuns'].sum().sort_values(ascending=False).head(10)

batsmanStrikerName
Virat Kohli         73432.0
Shikhar Dhawan      66736.0
Jos Buttler         65966.0
Shubman Gill        63139.0
Faf du Plessis      59753.0
KL Rahul            53761.0
David Warner        51636.0
Sanju Samson        49452.0
Suryakumar Yadav    48337.0
Ruturaj Gaikwad     48044.0
Name: batsmanStrikerRuns, dtype: float64

In [41]:
df3[df3['event'] == 'WICKET'].groupby('bowlerStrikerName').size().sort_values(ascending=False).head(10)

bowlerStrikerName
Rashid Khan         115
Harshal Patel        86
Yuzvendra Chahal     78
Mohammed Shami       76
Ravindra Jadeja      73
Arshdeep Singh       72
Kagiso Rabada        71
Sandeep Sharma       69
Jasprit Bumrah       68
Avesh Khan           66
dtype: int64

In [42]:
df3.head()

,series_name,match_id,match_name,status,venue,season,commText,event,batsmanStrikerName,batTeamName,batsmanStrikerRuns,bowlerStrikerName,batTeamScore,inningsId,ballNbr,complete_datetime
0,"Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, no run, short of a len...",NONE,Warner,SRH,0.0,Tymal Mills,0.0,1.0,1.0,2017-04-05 14:32:22.378
1,"Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, no run, very full and ...",NONE,Warner,SRH,0.0,Tymal Mills,0.0,1.0,2.0,2017-04-05 14:32:56.547
2,"Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, B0$, B1$, sublime timin...",FOUR,Warner,SRH,4.0,Tymal Mills,4.0,1.0,3.0,2017-04-05 14:33:36.255
3,"Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Warner, no run, short of a len...",NONE,Warner,SRH,4.0,Tymal Mills,4.0,1.0,4.0,2017-04-05 14:34:15.553
4,"Indian Premier League, 2017",18121,Sunrisers Hyderabad vs Royal Challengers Banga...,completed,"Rajiv Gandhi International Stadium, Hyderabad",2017,"Tymal Mills to Dhawan, no run, on a good leng...",NONE,Dhawan,SRH,0.0,Tymal Mills,6.0,1.0,5.0,2017-04-05 14:35:54.119
